cau 1:

In [1]:
from PIL import Image
import numpy as np
from scipy.ndimage import map_coordinates

# 1. Đọc ảnh kiwi gốc
image = Image.open('kiwi.jpg')  # <-- Đúng tên ảnh gốc ở đây!
image_np = np.array(image)

# 2. Tịnh tiến ảnh 50px sang phải, 30px xuống
height, width = image_np.shape[:2]
dx, dy = 50, 30
translated_image = np.zeros_like(image_np)
translated_image[dy:height, dx:width] = image_np[0:height - dy, 0:width - dx]

# 3. Hiệu ứng sóng (wave effect)
Y, X = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')
amplitude = 10
frequency = 2 * np.pi / 64
X_wave = X + amplitude * np.sin(frequency * Y)
Y_wave = Y

# 4. Biến đổi tọa độ
warped_image = np.zeros_like(translated_image)
for i in range(3):  # RGB
    warped_image[..., i] = map_coordinates(translated_image[..., i], [Y_wave, X_wave], order=1, mode='reflect')

# 5. Lưu kết quả
output = Image.fromarray(warped_image)
output.save('kiwi_wave.jpg')
print("✅ Ảnh đã được lưu thành công dưới tên 'kiwi_wave.jpg'")


✅ Ảnh đã được lưu thành công dưới tên 'kiwi_wave.jpg'


cau 2:

In [5]:
from PIL import Image
import numpy as np

def apply_gradient_color(image, color_start, color_end):
    """
    Áp dụng gradient màu lên đối tượng ảnh (theo trục dọc).
    `color_start` và `color_end` là tuple RGB.
    """
    image = image.convert("RGBA")
    arr = np.array(image)
    mask = arr[..., 3] > 0  # Lấy phần có alpha > 0 (đối tượng)

    height = arr.shape[0]
    gradient = np.linspace(np.array(color_start), np.array(color_end), height).astype(np.uint8)

    for y in range(height):
        arr[y, mask[y], :3] = gradient[y]

    return Image.fromarray(arr)

# 1. Đọc ảnh đu đủ và dưa hấu (ảnh PNG có nền trong hoặc đã được xóa nền)
papaya = Image.open('papaya.png').convert("RGBA")
watermelon = Image.open('watermelon.png').convert("RGBA")

# 2. Đổi màu:
# Papaya: đỏ → xanh lá (RGB: (255,0,0) → (0,255,0))
papaya_colored = apply_gradient_color(papaya, (255, 0, 0), (0, 255, 0))

# Watermelon: vàng → tím (RGB: (255,255,0) → (128, 0, 128))
watermelon_colored = apply_gradient_color(watermelon, (255, 255, 0), (128, 0, 128))

# 3. Tạo nền trong suốt đủ lớn để ghép cả 2 quả
canvas_width = papaya_colored.width + watermelon_colored.width
canvas_height = max(papaya_colored.height, watermelon_colored.height)
background = Image.new("RGBA", (canvas_width, canvas_height), (0, 0, 0, 0))

# 4. Ghép ảnh
background.paste(papaya_colored, (0, 0), papaya_colored)
background.paste(watermelon_colored, (papaya_colored.width, 0), watermelon_colored)

# 5. Lưu ảnh
background.save("fruits_combined.png")
print("✅ Đã lưu ảnh PNG với đu đủ và dưa hấu có gradient màu.")


✅ Đã lưu ảnh PNG với đu đủ và dưa hấu có gradient màu.


Cau 3:

In [7]:
from PIL import Image
import numpy as np
from scipy.ndimage import rotate

# 1. Đọc ảnh núi và thuyền
mountain = Image.open("mountain.jpg").convert("RGB")
boat = Image.open("boat.jpg").convert("RGB")

# 2. Resize cả hai ảnh về cùng kích thước (ví dụ 400x400)
target_size = (400, 400)
mountain = mountain.resize(target_size)
boat = boat.resize(target_size)

# 3. Xoay 45 độ (giữ kích thước gốc)
mountain_np = np.array(mountain)
boat_np = np.array(boat)

mountain_rotated = rotate(mountain_np, angle=45, reshape=False, mode='nearest')
boat_rotated = rotate(boat_np, angle=45, reshape=False, mode='nearest')

# 4. Phản chiếu dọc (vertical flip)
mountain_mirrored = np.flipud(mountain_rotated)
boat_mirrored = np.flipud(boat_rotated)

# 5. Tạo canvas trắng và ghép ảnh
h, w = mountain_mirrored.shape[:2]
canvas = np.ones((h, w * 2, 3), dtype=np.uint8) * 255  # Nền trắng

canvas[:, :w] = mountain_mirrored
canvas[:, w:] = boat_mirrored

# 6. Lưu kết quả
result = Image.fromarray(canvas)
result.save("mountain_boat_mirror.jpg")
print("✅ Đã lưu ảnh thành công: mountain_boat_mirror.jpg")


✅ Đã lưu ảnh thành công: mountain_boat_mirror.jpg


Cau 4:

In [1]:
from PIL import Image
import numpy as np
from scipy.ndimage import map_coordinates

# 1. Đọc ảnh ngôi chùa
image = Image.open("pagoda.jpg").convert("RGB")

# 2. Phóng to ảnh lên 5 lần
scale = 5
image_large = image.resize((image.width * scale, image.height * scale), resample=Image.BICUBIC)
image_np = np.array(image_large)

# 3. Áp dụng biến đổi hình học để tạo hiệu ứng "uốn cong"
height, width = image_np.shape[:2]

# Tạo lưới tọa độ
Y, X = np.meshgrid(np.arange(height), np.arange(width), indexing='ij')

# Biến đổi dạng "uốn cong" ngang theo sin trên trục dọc
# X lệch theo sin(Y)
amplitude = 30  # độ cong
frequency = 2 * np.pi / 200
X_warp = X + amplitude * np.sin(frequency * Y)
Y_warp = Y

# 4. Warping ảnh bằng map_coordinates
warped = np.zeros_like(image_np)
for i in range(3):  # RGB
    warped[..., i] = map_coordinates(image_np[..., i], [Y_warp, X_warp], order=1, mode='reflect')

# 5. Lưu ảnh kết quả
result = Image.fromarray(warped)
result.save("pagoda_warped.jpg")
print("✅ Đã lưu ảnh ngôi chùa đã uốn cong vào 'pagoda_warped.jpg'")


✅ Đã lưu ảnh ngôi chùa đã uốn cong vào 'pagoda_warped.jpg'


Cau 5:

In [7]:
from PIL import Image, ImageFilter
import numpy as np

def translate_image(image, x_offset, y_offset):
    return image.transform(image.size, Image.AFFINE, (1, 0, x_offset, 0, 1, y_offset))

def rotate_image(image, angle, reshape):
    return image.rotate(angle, expand=reshape)

def zoom_image(image, zoom_factor):
    width, height = image.size
    new_size = (int(width * zoom_factor), int(height * zoom_factor))
    return image.resize(new_size, Image.LANCZOS)

def gaussian_blur(image, sigma):
    return image.filter(ImageFilter.GaussianBlur(sigma))

def wave_transform(image, amplitude):
    img_array = np.array(image)
    rows, cols, channels = img_array.shape
    for i in range(rows):
        img_array[i] = np.roll(img_array[i], int(amplitude * np.sin(2 * np.pi * i / 20)), axis=0)
    return Image.fromarray(img_array)

def main():
    print("Chọn ảnh (1: image1.jpg, 2: image2.jpg, 3: image3.jpg):")
    choice = input("Nhập số (1, 2 hoặc 3): ")
    image_path = f"image{choice}.jpg"
    
    try:
        image = Image.open(image_path).convert("RGBA")
    except FileNotFoundError:
        print("Tệp không tìm thấy!")
        return

    while True:
        print("\nChọn phép biến đổi:")
        print("1: Tịnh tiến")
        print("2: Xoay")
        print("3: Phóng to/thu nhỏ")
        print("4: Làm mờ Gaussian")
        print("5: Biến đổi sóng")
        print("0: Thoát")
        
        option = input("Nhập số lựa chọn: ")
        
        if option == '1':
            x_offset = int(input("Nhập số pixel di chuyển theo x: "))
            y_offset = int(input("Nhập số pixel di chuyển theo y: "))
            image = translate_image(image, x_offset, y_offset)
        
        elif option == '2':
            angle = float(input("Nhập góc xoay (độ): "))
            reshape = input("Có muốn thay đổi kích thước không? (y/n): ") == 'y'
            image = rotate_image(image, angle, reshape)
        
        elif option == '3':
            zoom_factor = float(input("Nhập hệ số phóng to/thu nhỏ: "))
            image = zoom_image(image, zoom_factor)
        
        elif option == '4':
            sigma = float(input("Nhập giá trị sigma cho làm mờ Gaussian: "))
            image = gaussian_blur(image, sigma)
        
        elif option == '5':
            amplitude = float(input("Nhập biên độ sóng: "))
            image = wave_transform(image, amplitude)
        
        elif option == '0':
            break
        
        else:
            print("Lựa chọn không hợp lệ! Vui lòng thử lại.")
        
        # Lưu ảnh kết quả
        image.save("output_image.png")
        print("Đã lưu ảnh kết quả dưới tên 'output_image.png'.")

if __name__ == "__main__":
    main()

Chọn ảnh (1: image1.jpg, 2: image2.jpg, 3: image3.jpg):
Tệp không tìm thấy!
